In [1]:
import math

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

from torchtext import data
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

import torch.nn.utils.rnn as rnn

import pandas as pd


import copy
import time

import gc

In [11]:
# データの前処理（Word Embedding）
# 試作Ver

# テキストを単語で分割
tokenizer = get_tokenizer('basic_english')

# data field定義
TEXT_N  = data.Field(sequential=True, lower=True, tokenize=tokenizer, init_token='<cls>')
#SECTION = data.Field(sequential=False, use_vocab=False)
#TREND_N = data.Field(sequential=False, use_vocab=False)
#PRICE_N = data.Field(sequential=False, use_vocab=False)
LABEL = data.Field(sequential=False, use_vocab=False)


# CSVファイルを読み込み、TabularDatasetオブジェクトの作成
train_data, test_data = data.TabularDataset.splits(path ='tweet-transformer/1h',
                                                   train='test7_v2.csv',
                                                   test ='test7_v2.csv',
                                                   format='csv',
                                                   skip_header = True,
                                                   fields=[('tweet_n', TEXT_N),
                                                           #('section', SECTION),
                                                           #('trend_n', TREND_N),
                                                           #('price_n', PRICE_N),
                                                           ('label', LABEL)])
print("データ読み込み完了")

# 単語辞書の作成
TEXT_N.build_vocab(train_data, min_freq=2)
vocab = TEXT_N.vocab
print('辞書作成完了')

# テキストを数値ベクトル化、バッチに分割
train_iter, test_iter = data.BucketIterator.splits((train_data, test_data),
                                                   batch_sizes=(1024, 512))

# メモリ解放
gc.collect()
print('メモリ解放')

データ読み込み完了
辞書作成完了
メモリ解放


In [12]:
# パラメーター
ntokens = len(vocab)  # size of vocabulary
d_model = 512  # embedding dimension
nhead   = 8    # number of heads in nn.MultiheadAttention
d_hid   = 2048  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 6    # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
dropout = 0.2  # dropout probability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# Transformerモデルの概要
class TransformerModel(nn.Module):

    def __init__(self,
                 ntoken: int,
                 d_model: int,
                 nhead: int,
                 d_hid: int,
                 nlayers: int,
                 dropout: float = 0.5):

        super().__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model
        self.emb = nn.Embedding(ntoken, d_model, padding_idx=0)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.classifer = nn.Linear(d_model, 3)
        #self.logsoftmax = nn.LogSoftmax()

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.emb.weight.data.uniform_(-initrange, initrange)
        self.classifer.bias.data.zero_()
        self.classifer.weight.data.uniform_(-initrange, initrange)

    #データの流れ
    #def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
    def forward(self, src: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        print(src.shape)
        embedded = self.emb(src) * math.sqrt(self.d_model)
        print(embedded.shape)
        pos = self.pos_encoder(embedded)
        print(pos.shape)
        #encoder_out = self.transformer_encoder(pos, src_mask)
        encoder_out = self.transformer_encoder(pos)
        print(encoder_out.shape)
        x = encoder_out.mean(dim=1)
        print(x.shape)
        output = self.classifer(x)
        print(output.shape)
        #output = self.logsoftmax(output)
        return output
        

In [14]:
# Self Attention Mask
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [15]:
# PositionalEncodingの概要
class PositionalEncoding(nn.Module):

    def __init__(self,
                 d_model: int,
                 dropout: float = 0.1,
                 max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
bptt = 32
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [16]:
#訓練 & テスト

epochs = 10
model = TransformerModel(ntokens, d_model, nhead, d_hid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
lr = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

torch.manual_seed(0)

print("学習開始")
for epoch in range(epochs):
    print(f"------------------{epoch=}------------------")
    epoch_start_time = time.time()
    model.train()
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    for idx, batch in enumerate(iter(train_iter)):
        predictions = model(batch.tweet_n.to(device))
        labels = batch.label.to(device) - 1

        loss = criterion(predictions, labels)

        correct = predictions.argmax(axis=1) == labels
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        lr = scheduler.get_last_lr()[0]
        scheduler.step()
        
    with torch.no_grad():
        model.eval()
        test_epoch_loss = 0
        test_epoch_correct = 0
        test_epoch_count = 0

        for idx, batch in enumerate(iter(test_iter)):
            predictions = model(batch.text.to(device))
            labels = batch.label.to(device) - 1
            test_loss = criterion(predictions, labels)

            correct = predictions.argmax(axis=1) == labels
            acc = correct.sum().item() / correct.size(0)

            test_epoch_correct += correct.sum().item()
            test_epoch_count += correct.size(0)
            test_epoch_loss += loss.item()
            
    elapsed = time.time() - epoch_start_time

    print(f"{epoch_loss=}")
    print(f"epoch_accuracy: {epoch_correct / epoch_count}")
    print(f"{test_epoch_loss=}")
    print(f"test_epoch_accuracy: {test_epoch_correct / test_epoch_count}")
    print(f'time: {elapsed:5.2f}s')

学習開始
------------------epoch=0------------------
torch.Size([90, 1024])
torch.Size([90, 1024, 512])
torch.Size([90, 1024, 512])
torch.Size([90, 1024, 512])
torch.Size([90, 512])
torch.Size([90, 3])


ValueError: Expected input batch_size (90) to match target batch_size (1024).

In [25]:
gc.collect()

8

In [22]:
train_ = next(iter(train_iter))
print(train_)


[torchtext.legacy.data.batch.Batch of size 64]
	[.tweet_n]:[torch.LongTensor of size 63x64]
	[.label]:[torch.LongTensor of size 64]


In [11]:
train_iter

In [12]:
train_data

In [ ]:
'''
l = 0
t = 0
for batch in train_iter:
    l = max(l, batch.label.shape)
    t = max(t, batch.tweet_n.shape)
print(l,t)
'''


for batch in train_iter:
    print(batch.label.shape)
    print(batch.tweet_n.shape)



In [10]:
i=0
for example in train_data:
    print(example.label)
    print(example.tweet_n)
    i+=1
    if i==10: break

0
['top50', 'cryptocurrency', 'in/out', 'update', 'last', '12', 'hours', '(', 'crypto', 'bitcoin', 'altcoin', ')', 'in', 'out']
0
['bitcoin', '.', '39💚', '+159', '.', '43', 'last', '1', 'hour', '(', '+0', '.', '46%', ')', '💚', '+219', '.', '3', 'last', '5', 'hours', '(', '+0', '.', '63%', ')', '💔', '-845', '.', '02', 'last', '24', 'hours', '(', '-2', '.', '35%', ')', 'bitcoinpriceupdates', 'bitcoin', 'hourlycrypto', 'cryptoupdates', 'cryptopowered', 'api']
0
['bitcoincurrent', 'price', '35060€', '29561', '.', '01cryptocurrencies', 'blockchain', 'btc']
0
['gold', '|', '|', 'gold', 'waylong', 'short', 'btc', 'trade8']
0
['binance', 'activity', '🔵', 'bought', 'worth', '47', '.', '1m+', 'usdt🔴', 'sold', 'worth', '30', '.', '1m+', 'usdt🔴', 'sold', 'worth', '28', '.', '4m+', 'usdt🔵', 'bought', 'worth', '23', '.', '9m+', 'usdtcryptocurrency', 'cryptotrading', 'binance', '-']
0
['price', 'ofspaghetti', 'alla', 'carbonarais', '31', ',', '359', 'satoshisor', '0', '.', '00031359', 'btc1', 'btc', 

In [8]:
i=0
for idx, batch in enumerate(iter(train_iter)):
    print(idx)
    print(batch)
    i+=1
    if i==5: break

0

[torchtext.legacy.data.batch.Batch of size 64]
	[.tweet_n]:[torch.LongTensor of size 49x64]
	[.label]:[torch.LongTensor of size 64]
1

[torchtext.legacy.data.batch.Batch of size 64]
	[.tweet_n]:[torch.LongTensor of size 45x64]
	[.label]:[torch.LongTensor of size 64]
2

[torchtext.legacy.data.batch.Batch of size 64]
	[.tweet_n]:[torch.LongTensor of size 79x64]
	[.label]:[torch.LongTensor of size 64]
3

[torchtext.legacy.data.batch.Batch of size 64]
	[.tweet_n]:[torch.LongTensor of size 57x64]
	[.label]:[torch.LongTensor of size 64]
4

[torchtext.legacy.data.batch.Batch of size 64]
	[.tweet_n]:[torch.LongTensor of size 63x64]
	[.label]:[torch.LongTensor of size 64]


In [20]:
rnn.pad_sequence(train_iter).size()

TypeError: pad_sequence(): argument 'sequences' (position 1) must be tuple of Tensors, not BucketIterator